In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [33]:
df= pd.read_csv("ngn_eur_data.csv")

In [34]:
df1 = df.copy()

In [35]:
df1 = df1.dropna()

In [36]:
df1.isnull().sum()

Unnamed: 0              0
EUR_NGN_1. open         0
EUR_NGN_2. high         0
EUR_NGN_3. low          0
EUR_NGN_4. close        0
TRENDS_Euro to Naira    0
dtype: int64

In [37]:
# Keep only the needed columns
# Keep only the needed columns
df1 = df1[['Unnamed: 0', 'EUR_NGN_4. close', 'TRENDS_Euro to Naira']]

#  Rename columns
df1.rename(columns={
    'Unnamed: 0': 'Date',
    'EUR_NGN_4. close': 'NGN_EURO',
    'TRENDS_Euro to Naira': 'Google_Trends'
}, inplace=True)


print(df1.head())

          Date   NGN_EURO  Google_Trends
16  2014-12-01  229.63000           15.0
17  2014-12-02  222.48000           15.0
18  2014-12-03  220.37000           15.0
19  2014-12-04  221.64000           15.0
20  2014-12-05  220.28999           15.0


In [7]:
df1.head()

,Date,EUR_NGN_1. open,EUR_NGN_2. high,EUR_NGN_3. low,NGN_EURO,TRENDS_Euro to Naira
16,2014-12-01,222.37,230.14000,222.07001,229.63000,15.0
17,2014-12-02,229.62,229.67000,222.36000,222.48000,15.0
18,2014-12-03,222.48,222.59000,220.27000,220.37000,15.0
19,2014-12-04,220.36,224.03999,220.28000,221.64000,15.0
20,2014-12-05,221.63,225.00000,220.28000,220.28999,15.0


In [8]:
df1.shape

(2445, 6)

In [38]:
# Prepare the data
# Your dataframe is already named df
features = df1[['NGN_EURO', 'Google_Trends']]

# Scale features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i, 0])  # NGN_EURO is first column
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_features, seq_length)

# Train/Test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [39]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')


C:\Users\kefa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 0.0040 - val_loss: 0.0631
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 3.2974e-04 - val_loss: 0.0121
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 2.4294e-04 - val_loss: 0.0169
Epoch 4/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - loss: 1.9087e-04 - val_loss: 0.0127
Epoch 5/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 1.5379e-04 - val_loss: 0.0148
Epoch 6/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 1.4083e-04 - val_loss: 0.0198
Epoch 7/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 1.4745e-04 - val_loss: 0.0148
Epoch 8/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 1.2144e-04 - val_loss: 0.0164
Epoch 9/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 1.1039e-04 - val_loss: 0.0178
Epoch 10/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - loss: 1.1582e-04 - val_loss: 0.0184
Epoch 11/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 1.0306e-04 - val_loss: 0.0193
Epoch 12/50
60/60 ━━━━━

In [ ]:
#prediction function

In [41]:
def predict_until_date(model, last_sequence, target_date, scaler):
    today = datetime.today().date()
    target_date = datetime.strptime(target_date, "%Y-%m-%d").date()

    # Calculate how many days ahead
    future_days = (target_date - today).days
    
    if future_days <= 0:
        raise ValueError("Target date must be in the future.")

    predictions = []
    current_sequence = last_sequence.copy()

    for _ in range(future_days):
        input_seq = np.expand_dims(current_sequence, axis=0)
        pred_scaled = model.predict(input_seq, verbose=0)
        
        predictions.append(pred_scaled[0, 0])
        
        next_step = np.hstack((pred_scaled, np.array([[current_sequence[-1, 1]]])))
        current_sequence = np.vstack((current_sequence[1:], next_step))

    # Inverse transform
    predictions_array = np.array(predictions).reshape(-1, 1)
    zeros_array = np.zeros((predictions_array.shape[0], 1))
    inv_predictions = scaler.inverse_transform(np.hstack((predictions_array, zeros_array)))[:, 0]

    # Final prediction for target date
    final_price = inv_predictions[-1]
    return final_price


In [ ]:
# Usage Example

In [42]:
# Assume model is trained already and last_sequence is ready

# User inputs
target_date = input("Enter the future date (YYYY-MM-DD): ")

# Make prediction
try:
    last_sequence = scaled_features[-seq_length:]  # Last 60 records
    predicted_price = predict_until_date(model, last_sequence, target_date, scaler)
    print(f"Predicted NGN/EURO price on {target_date}: {predicted_price:.2f}")
except Exception as e:
    print(f"Error: {e}")


Enter the future date (YYYY-MM-DD):  2025-04-29


Predicted NGN/EURO price on 2025-04-29: 1675.44
